In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from pylab import cm
import mpltex
import os
from os import listdir
from os.path import isfile, join
import itertools
from matplotlib import pyplot
from scipy import stats

import re

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Avenir'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.linewidth'] = 2

In [ ]:
current_path = os.getcwd()
mypath = current_path
ibs = []

for f in listdir(mypath):
    if f[:6] == "result":
        print(f)
        filename = os.path.splitext(f)[0]
        t = filename.split('_')
        ibs.append(t[2])

ibs = sorted(ibs)
ibs_lst = []
for ib in ibs:
    ibs_lst.append(ib[1:-1])

In [ ]:
ib_dict = {}

for ib in ibs:
    file = open(f"result_0.5_{ib}.pkl",'rb')
    object_file = pickle.load(file)
    file.close()
    temp = {}
    for metric, metric_dict in object_file.items():
        temp[metric] = {}
        for policy_omega, policy_omega_dict in metric_dict.items():
            if policy_omega.split('_')[0] == "No investigation":
                renamed_policy_omega = "No Investigation_"+policy_omega.split('_')[1]
                temp[metric][renamed_policy_omega] = policy_omega_dict
            elif policy_omega.split('_')[0] == "LowFP":
                renamed_policy_omega = "MinFP_"+policy_omega.split('_')[1]
                temp[metric][renamed_policy_omega] = policy_omega_dict
            else:
                temp[metric][policy_omega] = policy_omega_dict
    ib_dict[ib] = temp

In [ ]:
policies = ["No Investigation", "Bayes", "MaxEntropy", "MinFP", "All"]
policies = sorted(policies)

omegas = [omega.split('_')[1] for omega in ib_dict[ib]['MSE'].keys()]
omegas = sorted(np.unique(omegas))

In [ ]:
print(omegas)

fp_dict[fp_rate][metric][Policy_omega]
fp_dict[0.1]['MSE']['Raw_1'].keys()

In [ ]:
result_dict = {}
for omega in omegas:
    result_dict[omega] = {}
    for policy in policies:
        result_dict[omega][policy] = []

In [ ]:
for omega, omega_dict in result_dict.items():
    for policy, policy_omega_list in omega_dict.items():
        name = policy+'_'+str(omega)
        for ib in ib_dict:
            nptemp = np.zeros(ib_dict[ib]['MSE'][name].shape[0]+1)
            nptemp[0] = ib[1:-1]
            nptemp[1:] = ib_dict[ib]['MSE'][name][:,-1]
            result_dict[omega][policy].append(nptemp)

for omega, omega_dict in result_dict.items():
    for policy, policy_omega_list in omega_dict.items():
        name = policy+'_'+str(omega)
        for ib in ib_dict:
            result_dict[omega][policy] = np.array(result_dict[omega][policy])

In [ ]:
np.array(result_dict[omega][policy]).shape

In [ ]:
diff_fp_mse_dict = {}
for ib, single_ib_dict in ib_dict.items():
    for policy, policy_dict in single_ib_dict['MSE'].items():
        try:
            diff_fp_mse_dict[policy.split('_')[0]].append(policy_dict.mean(axis=0)[-1])
        except:
            diff_fp_mse_dict[policy.split('_')[0]] = []
            diff_fp_mse_dict[policy.split('_')[0]].append(policy_dict.mean(axis=0)[-1])


In [ ]:
ib_lst =[]
for ib in ibs:
    ib_lst.append(int(ib[1:-1]))
fig, axes = plt.subplots(nrows = 1, ncols=1, figsize=(10,5))
c = ['black','brown']
locs = ["upper left", "upper center"]

l = ['-', '--']
m = ['o','^', 'x', 's',  'd']
c = ['black', 'brown']
cc = itertools.cycle(c)
ll = itertools.cycle(l)
plot_lines = []
for omega in omegas[::-1]:
    print(omega)
    j = 0
    each_omega = []
    c = next(cc)
    l = next(ll)
    for policy in policies:
        if policy == "No Investigation":
            x = result_dict[omega][policy][:,0]
            y = result_dict[omega][policy][:,1:]
            y_mean = y.mean(axis=1)
            axes.plot(ib_lst, y_mean, l, color=c, marker = m[j%len(m)], linewidth=1)
        else: 
            x = result_dict[omega][policy][:,0]
            y = result_dict[omega][policy][:,1:]
            y_mean = y.mean(axis=1)
            axes.plot(ib_lst, y_mean, l, color=c, marker = m[j%len(m)], linewidth=1)
        j += 1
    # plot_lines.append(each_omega)

axes.set_ylim([0,0.8])
lines = axes.get_lines()
legend1 = pyplot.legend([lines[i] for i in [0,1,2,3,4,5]], policies, ncol = 3, loc="upper center")
axes.add_artist(legend1)
for l in lines:
    l.set_marker('')
    l.set_linestyle('-')
legend2 = pyplot.legend([lines[i] for i in [0,6] ], ["$\omega$=0", "$\omega$=0.05"], loc=(0.72,0.4))
axes.add_artist(legend2)
for i, l in enumerate(lines):
    l.set_marker(m[i%len(m)])

for i, l in enumerate(lines):
    if i>4:
        l.set_linestyle('--')
axes.set_ylabel('MSE', labelpad=10)
axes.set_xlabel('Investigation Budget ($I$)', labelpad=10)



up_marker = ['_', "1"]
down_marker = ['_', "2"]
l = ['-', ':']
mmm = ['o','^', 'x', 's', 'd']
c = ['black', 'brown']
cc = itertools.cycle(c)
ll = itertools.cycle(l)
mm = itertools.cycle(mmm)
for omega in omegas[::-1]:
    c = next(cc)
    l = next(ll)
    up = itertools.cycle(up_marker)
    down = itertools.cycle(down_marker)
    for policy in policies:
        u = next(up)    
        d = next(down)
        if policy != 'Random':
            val = result_dict[omega][policy]
            mean = val[:,1:].mean(axis=1)
            print(policy, mean)
            std = val[:,1:].std(axis = 1)
            n= val.shape[1]
            yerr = std / np.sqrt(n) * stats.t.ppf(1-0.05/2, n - 1)
            x = val[:,0]
            bar1 = plt.errorbar(x, 
                                y=mean, 
                                linestyle='', 
                                yerr = yerr,                         
                                # label=f'{policy}', 
                                # marker=m, 
                                color=c, 
                                # markevery=10, 
                                capsize=0
                                )
            plt.plot(x,mean+yerr, marker=u, ls="", color=c, ms=10)
            plt.plot(x,mean-yerr, marker=d, ls="", color=c, ms=10)
fig.savefig('investigation_budget.eps', format='eps', bbox_inches='tight')
plt.show()

